# MBTA database connection and data extraction
## Location data

In [7]:
library(DBI)
# Input the credential files for accessing database
CERT = "D:/MBTA/movement/postgresql/zhuohan.crt"
KEY = "D:/MBTA/movement/postgresql/zhuohan.key"
ROOTCERT = "D:/MBTA/movement/postgresql/root-ca.crt"
USER = "zhuohan"
PASSWORD = "5irmwhg382aae"
con <- dbConnect(RPostgres::Postgres(), host="mbta01.mghpcc.org", port="5432", dbname = "mbta_dw",
                     sslcert = CERT, sslkey = KEY, sslrootcert = ROOTCERT,
                     sslmode="verify-full", user = USER, password = PASSWORD)
# Input months and years here
MONTHS_TO_EXTRACT = c('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12')
YEARS_TO_EXTRACT = c('19')

In [5]:
# Extract the data as given month,year and rail type
query_location_table = function(month, year, rail_type){
    query = dbSendQuery(con,statement = paste("SELECT *","FROM","ttr.",
                                           paste(rail_type, "location", year, month, sep = "",collapse = "")))
    return(query)
}
read_tables_to_file <- function(month, year) {
    for (type in c("lightrail", "heavyrail")) {
        rs = query_location_table(month, year, type)
        df = dbFetch(rs, n = -1)
        filename = paste(type, "trajectories", month, year, ".csv", sep = "-", collapse = "")
        write.csv(df, file.path("../../data/raw/vehicle-location", filename), row.names = FALSE)
        dbClearResult(rs)
        }
}

In [6]:
extract_locations_by_month_year <- function(monthlist, yearlist) {
    for (yy in yearlist) {
        for (mm in monthlist){
            read_tables_to_file(mm, yy)
        }
    }
    dbDisconnect(con)     #close connetion
}


In [10]:
extract_locations_by_month_year(MONTHS_TO_EXTRACT, YEARS_TO_EXTRACT)

Warning message in result_create(conn@ptr, statement):
"Closing open result set, cancelling previous query"


In [ ]:
# if (!interactive()) {
#   extract_locations_by_month_year(MONTHS_TO_EXTRACT, YEARS_TO_EXTRACT)
# }